### Instalação e importação das bibliotecas

In [9]:
!pip install scikit-learn pandas numpy

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error

ModuleNotFoundError: No module named 'numpy.typing'

### Carregamento dos Dados

In [ ]:
# Carregar os dados
ratings = pd.read_csv('u.data', sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'])
movies = pd.read_csv('u.item', sep='|', names=['item_id', 'title'], usecols=[0, 1], encoding='latin-1')
users = pd.read_csv('u.user', sep='|', names=['user_id', 'age', 'gender', 'occupation', 'zip_code'])

# Exibir as primeiras linhas dos DataFrames
print(ratings.head())
print(movies.head())
print(users.head())

### Divisão dos Dados

In [ ]:
# Dividir os dados em 80% para treinamento e 20% para validação
train_data, test_data = train_test_split(ratings, test_size=0.2, random_state=42)

# Criar a matriz de usuário-item para o conjunto de treinamento
train_matrix = train_data.pivot(index='user_id', columns='item_id', values='rating').fillna(0)
test_matrix = test_data.pivot(index='user_id', columns='item_id', values='rating').fillna(0)

### Treinamento do Modelo

In [ ]:
# Calcular a similaridade do cosseno entre os usuários
user_similarity = cosine_similarity(train_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=train_matrix.index, columns=train_matrix.index)
print(user_similarity_df.head())

### Validação do Modelo

In [ ]:
# Função para prever a nota de um usuário para um item
def predict_rating(user_id, item_id, train_matrix, user_similarity_df):
    # Obter a similaridade do usuário com todos os outros usuários
    user_similarities = user_similarity_df[user_id]
    
    # Obter as notas dos outros usuários para o item
    item_ratings = train_matrix[item_id]
    
    # Calcular a nota prevista
    weighted_sum = np.dot(user_similarities, item_ratings)
    sum_of_weights = np.sum(user_similarities)
    
    if sum_of_weights == 0:
        return 0
    else:
        return weighted_sum / sum_of_weights

# Prever as notas para o conjunto de validação
test_data['predicted_rating'] = test_data.apply(lambda row: predict_rating(row['user_id'], row['item_id'], train_matrix, user_similarity_df), axis=1)

# Calcular o RMSE
rmse = np.sqrt(mean_squared_error(test_data['rating'], test_data['predicted_rating']))
print(f'RMSE: {rmse}')

### Recomendação de Filmes

In [ ]:
# Função para recomendar filmes
def recomendar(user_id, train_matrix, user_similarity_df, n_recommendations=5):
    # Obter todos os itens
    all_items = train_matrix.columns
    
    # Prever a nota para todos os itens
    predictions = [predict_rating(user_id, item_id, train_matrix, user_similarity_df) for item_id in all_items]
    
    # Criar um DataFrame com as previsões
    predictions_df = pd.DataFrame({'item_id': all_items, 'predicted_rating': predictions})
    
    # Ordenar as previsões por nota
    predictions_df = predictions_df.sort_values(by='predicted_rating', ascending=False)
    
    # Retornar os top N itens recomendados
    return predictions_df.head(n_recommendations)

# Exemplo de recomendação para um usuário específico
user_id = 1
recommendations = recomendar(user_id, train_matrix, user_similarity_df)
print(f'Recomendações para o usuário {user_id}:')
print(recommendations)